In [ ]:
#still in progress

# TO USE

### Run  Every cell, skip 4, run testing area
### Export dated_all_tables using Pickle
### New file created in directory

In [9]:
#Trevor O'Hearn
#8/4/20
#Written to Quickly Scrape the overview table of the Federal Reserve

import pandas as pd
import requests
from bs4 import BeautifulSoup
import WebScrapingMethods as wsm

In [10]:
base_url = 'https://www.federalreserve.gov/releases/h41/'

#get list for links
soup = wsm.getSoup(base_url)
hrefs = soup.select('.col-xs-1 a')
end_urls = []
for a in hrefs:
    #get links for 2020 and 2019
    if (a.attrs['href'][:4] == '2020' or a.attrs['href'][:4] == '2019'
        or a.attrs['href'][:4] == 'curr'):
        end_urls.append(a.attrs['href'] + '/h41.htm')
    else:
        break

urls = wsm.getLinks(base_url, end_url_list=end_urls)

In [11]:
url = urls[0]
bs = wsm.getSoup(url)
date = bs.select('.H41Release td p')[0].text.strip()
tables = bs.select('table')

In [12]:
bs = wsm.getSoup(urls[0])
table_columns = []
table_rows = []
table_data = {}
features = wsm.cleanFeatures(wsm.getFeatures(bs))
k = 0
failures = []
for table in tables:
    #get dimensions of table
    try:
        
        k += 1
        flag = 0
        if (table.select('th', limit=1)[0].attrs['id'].split('c')[1] == '0'):
            print('actually a table: {}'.format(table.select('th', limit=1)[0].text))
            flag = 1
        across = len(table.select('tr')[-1].select('p')) - 1 #can't count the header column
        down = len(table.select('td')) // across
        bank_bool = False
        if (across > 10): #table of banks
            bank_bool = True
        table_columns.append(across)
        table_rows.append(down)
        
        #get features
        table_num = int(table.select('tr th', limit=1)[0].attrs['id'].strip('t').split('c')[0])
        features[table_num]
        flag = 2
        #get data
        tds = table.select('td')
        data = []
        i = 0
        while i < down:
            text = None
            if (bank_bool): #total is in first column
                text = tds[across * i].text.strip()
            else: #total is last column
                text = tds[across * i + across - 1].text.strip()
            if (len(text) > 0):
                data.append(text)
            i += 1
        table_data[table_num] = dict(zip(features[table_num], data))
        flag = 3
        print('zipped : {}, {}'.format(table_num, flag))
    except:
        print(flag)
        failures.append(k)
        continue
print(failures)
#supposed to fail : 0,3,5, 7, 9, 11, 14, 17, 19 (0 indexed)

#take dictionaries of tables and make a singulare dictionary
all_tables_data = {}
for key in table_data:
    for k in table_data[key]:
        if k in all_tables_data.keys(): #key name already exists
            all_tables_data['{} in {}'.format(k, key)] = table_data[key][k]
        else:
            all_tables_data[k] = table_data[key][k]


#take singular dictionary and create dataframe
df = pd.DataFrame(data=all_tables_data, index=[0])


0
actually a table: Reserve Bank credit, related items, andreserve balances of depository institutions at Federal Reserve Banks
zipped : 2, 3
actually a table: Reserve Bank credit, related items, and reserve balances of depository institutions atFederal Reserve Banks
zipped : 3, 3
0
actually a table: Memorandum item
zipped : 5, 3
0
actually a table: Remaining Maturity
zipped : 7, 3
0
actually a table: Account name
zipped : 9, 3
0
actually a table: Credit Facilities LLCs:
zipped : 11, 3
0
actually a table: Assets, liabilities, and capital
zipped : 13, 3
actually a table: Assets, liabilities, and capital
zipped : 14, 3
0
actually a table: Assets, liabilities, and capital
zipped : 16, 3
actually a table: Assets, liabilities, and capital
zipped : 17, 3
0
0
actually a table: Federal Reserve notes and collateral
zipped : 20, 3
0
[1, 4, 6, 8, 10, 12, 15, 18, 19, 21]


# Clean table data dictionary

In [13]:
for k in table_data:
    for key in table_data[k]:
        val = table_data[k][key]
        val = wsm.removeUnicode(val)
        val = wsm.removePlus(val)
        val = wsm.removeComma(val)
        val = wsm.removeParentheses(val)
        try:
            val = int(val)
        except:
            print('int fail')
            continue
        table_data[k][key] = val
    

# DICTIONARY EXPANDED TO ALL URLS

In [12]:
#for url in urls:
    

84

# All URLS INTO DFS

In [12]:

dfs = []
for url in urls:
    bs = wsm.getSoup(url)
    date = bs.select('.H41Release td p')[0].text.strip()
    print(date)
    features = wsm.getFeatures(bs)
    table = bs.select('table')[1]
    tds = table.select('td')
    data = []
    i = 0
    while i < (len(tds) / 4):
        text = tds[4 * i + 3].text.strip()
        if (len(text) > 0):
            data.append(text)
        i += 1
    cleaned_features = wsm.cleanFeatures(features)

    dfvals = {}
    dfvals['Date'] = date
    for i, f in enumerate(cleaned_features[2]):
        dfvals[f] = data[i]

    df = pd.DataFrame(data=dfvals, index=[0])
    dfs.append(df)

#clean dataframe
df = dfs[0]
for d in dfs[1:]:
    df = df.append(d, ignore_index=False)
df['Date'] = pd.to_datetime(df['Date'], format='%B %d, %Y', errors='ignore')
df.set_index('Date', inplace=True)

all_dfs = [df]
columnNames = df.columns
df.fillna('0', inplace=True)
#clean data frame
for df in all_dfs:
    for c in columnNames:
        df[c] = df[c].apply(wsm.removeUnicode)
        df[c] = df[c].apply(wsm.removePlus)
        df[c] = df[c].apply(wsm.removeComma)
        df[c] = df[c].astype(int)
#df_all.to_csv('quickscrape.csv')


July 2, 2020
July 9, 2020
July 16, 2020
July 23, 2020
July 30, 2020
June 4, 2020
June 11, 2020
June 18, 2020
June 25, 2020
May 7, 2020
May 14, 2020
May 21, 2020
May 28, 2020
April 2, 2020
April 9, 2020
April 16, 2020
April 23, 2020
April 30, 2020
March 5, 2020
March 12, 2020
March 19, 2020
March 26, 2020
February 6, 2020
February 13, 2020
February 20, 2020
February 27, 2020
January 2, 2020
January 9, 2020
January 16, 2020
January 23, 2020
January 30, 2020
December 5, 2019
December 12, 2019
December 19, 2019
December 26, 2019
November 7, 2019
November 14, 2019
November 21, 2019
November 29, 2019
October 3, 2019
October 10, 2019
October 17, 2019
October 24, 2019
October 31, 2019
September 5, 2019
September 12, 2019
September 19, 2019
September 26, 2019
August 1, 2019
August 8, 2019
August 15, 2019
August 22, 2019
August 29, 2019
July 5, 2019
July 11, 2019
July 18, 2019
July 25, 2019
June 6, 2019
June 13, 2019
June 20, 2019
June 27, 2019
May 2, 2019
May 9, 2019
May 16, 2019
May 23, 2019
M

In [4]:
df = pd.DataFrame(data=dfvals, index=[0])

# Testing Area

In [15]:
dated_all_tables = {}
for url in urls:
    bs = wsm.getSoup(url)
    date = bs.select('.H41Release td p')[0].text.strip()
    table_columns = []
    table_rows = []
    table_data = {}
    features = wsm.cleanFeatures(wsm.getFeatures(bs))
    k = 0
    failures = []
    for table in tables:
        #get dimensions of table
        try:

            k += 1
            flag = 0
            if (table.select('th', limit=1)[0].attrs['id'].split('c')[1] == '0'):
#                print('actually a table: {}'.format(table.select('th', limit=1)[0].text))
                flag = 1
            across = len(table.select('tr')[-1].select('p')) - 1 #can't count the header column
            down = len(table.select('td')) // across
            bank_bool = False
            if (across > 10): #table of banks
                bank_bool = True
            table_columns.append(across)
            table_rows.append(down)

            #get features
            table_num = int(table.select('tr th', limit=1)[0].attrs['id'].strip('t').split('c')[0])
            features[table_num]
            flag = 2
            #get data
            tds = table.select('td')
            data = []
            i = 0
            while i < down:
                text = None
                if (bank_bool): #total is in first column
                    text = tds[across * i].text.strip()
                else: #total is last column
                    text = tds[across * i + across - 1].text.strip()
                if (len(text) > 0):
                    data.append(text)
                i += 1
            table_data[table_num] = dict(zip(features[table_num], data))
            flag = 3
#            print('zipped : {}, {}'.format(table_num, flag))
        except:
#            print(flag)
            failures.append(k)
            continue
#    print(failures)
    #supposed to fail : 0,3,5, 7, 9, 11, 14, 17, 19 (0 indexed)

    #take dictionaries of tables and make a singulare dictionary
    all_tables_data = {}
    for key in table_data:
        for k in table_data[key]:
            if k in all_tables_data.keys(): #key name already exists
                all_tables_data['{} in {}'.format(k, key)] = table_data[key][k]
            else:
                all_tables_data[k] = table_data[key][k]
    for k in table_data:
        for key in table_data[k]:
            val = table_data[k][key]
            val = wsm.removeUnicode(val)
            val = wsm.removePlus(val)
            val = wsm.removeComma(val)
            val = wsm.removeParentheses(val)
            try:
                val = int(val)
            except:
                continue
            table_data[k][key] = val
    dated_all_tables[date] = all_tables_data
    print(date)
    #take singular dictionary and create dataframe
    #df = pd.DataFrame(data=all_tables_data, index=[0])



August 6, 2020
July 2, 2020
July 9, 2020
July 16, 2020
July 23, 2020
July 30, 2020
June 4, 2020
June 11, 2020
June 18, 2020
June 25, 2020
May 7, 2020
May 14, 2020
May 21, 2020
May 28, 2020
April 2, 2020
April 9, 2020
April 16, 2020
April 23, 2020
April 30, 2020
March 5, 2020
March 12, 2020
March 19, 2020
March 26, 2020
February 6, 2020
February 13, 2020
February 20, 2020
February 27, 2020
January 2, 2020
January 9, 2020
January 16, 2020
January 23, 2020
January 30, 2020
December 5, 2019
December 12, 2019
December 19, 2019
December 26, 2019
November 7, 2019
November 14, 2019
November 21, 2019
November 29, 2019
October 3, 2019
October 10, 2019
October 17, 2019
October 24, 2019
October 31, 2019
September 5, 2019
September 12, 2019
September 19, 2019
September 26, 2019
August 1, 2019
August 8, 2019
August 15, 2019
August 22, 2019
August 29, 2019
July 5, 2019
July 11, 2019
July 18, 2019
July 25, 2019
June 6, 2019
June 13, 2019
June 20, 2019
June 27, 2019
May 2, 2019
May 9, 2019
May 16, 2019

In [19]:
dated_all_tables

162

# Export with Pickle

In [20]:
import pickle as p

p.dump( dated_all_tables, open( "dated_all_tables.p", "wb" ) )

# To Import

In [ ]:
#import pickle as p
#p.load( open( "dated_all_tables.p", "rb" ) )

# dated_all_tables to DF

In [39]:
df = pd.DataFrame(data=dated_all_tables).T
#if you want to fill NaN
df.fillna(0, inplace=True)
df.head()

,Reserve Bank credit,Securities held outright1,U.S. Treasury securities,Bills2,"Notes and bonds, nominal2","Notes and bonds, inflation-indexed2",Inflation compensation3,Federal agency debt securities2,Mortgage-backed securities4,Unamortized premiums on securities held outright5,...,Other liabilities and accrued dividends13,Less: Notes held by F.R. Banks,Reverse repurchase agreements11 in 16,Term deposits held by depository institutions in 16,Other deposits held by depository institutions in 16,Other12 in 16,Deferred availability cash items in 16,Earnings remittances due to the U.S. Treasury13,"Total factors, other than reserve balances, absorbing reserve \r\n funds",Securities held in custody for foreign official and international \r\n accounts
"August 6, 2020","6,905,429","6,241,464","4,305,651","326,044","3,670,435","273,306","35,866","2,347","1,933,466","321,669",...,0,0,0,0,0,0,0,0,0,0
"July 2, 2020","6,905,429","6,241,464","4,305,651","326,044","3,670,435","273,306","35,866","2,347","1,933,466","321,669",...,0,0,0,0,0,0,0,0,0,0
"July 9, 2020","6,905,429","6,241,464","4,305,651","326,044","3,670,435","273,306","35,866","2,347","1,933,466","321,669",...,0,0,0,0,0,0,0,0,0,0
"July 16, 2020","6,905,429","6,241,464","4,305,651","326,044","3,670,435","273,306","35,866","2,347","1,933,466","321,669",...,0,0,0,0,0,0,0,0,0,0
"July 23, 2020","6,905,429","6,241,464","4,305,651","326,044","3,670,435","273,306","35,866","2,347","1,933,466","321,669",...,0,0,0,0,0,0,0,0,0,0
